### Example Risk analysis in critical systems with multiple experts

Let \(S=\{1,2,3\}\) be a set of system components, and consider two experts \(E_1,E_2\) whose criteria about the components are modeled by the following fuzzy measures:

| **$A\subseteq S$** | **$\varnothing$** | **$\{1\}$** | **$\{2\}$** | **$\{3\}$** | **$\{1,2\}$** | **$\{1,3\}$** | **$\{2,3\}$** | **$\{1,2,3\}$** |
|--------------------|-------------------|-------------|-------------|-------------|---------------|---------------|---------------|-----------------|
| **$\mu_1(A)$**     | 0                 | 0.50        | 0.30        | 0.40        | 0.60          | 0.80          | 0.70          | 1.00            |
| **$\mu_2(A)$**     | 0                 | 0.20        | 0.10        | 0.20        | 0.30          | 0.40          | 0.35          | 1.00            |


In [5]:
def mu1(n_set_ids: list, coalition_ids: list, original) -> float:
    if len(coalition_ids) == 0:
        return 0.0
    coalition_str = "{" + ",".join(
        str(i+1)
        for i in sorted(coalition_ids)
    ) + "}"
    if coalition_str == "{}":
        return 0.0
    if coalition_str == "{1,2,3}":
        return 1.0
    if coalition_str == "{1}":
        return 0.5
    if coalition_str == "{2}":
        return 0.3
    if coalition_str == "{3}":
        return 0.4
    if coalition_str == "{1,2}":
        return 0.6
    if coalition_str == "{1,3}":
        return 0.8
    if coalition_str == "{2,3}":
        return 0.7
    raise ValueError("Coalition not found")


def mu2(n_set_ids: list, coalition_ids: list, original) -> float:
    if len(coalition_ids) == 0:
        return 0.0
    coalition_str = "{" + ",".join(
        str(i+1)
        for i in sorted(coalition_ids)
    ) + "}"
    if coalition_str == "{}":
        return 0.0
    if coalition_str == "{1,2,3}":
        return 1.0
    if coalition_str == "{1}":
        return 0.2
    if coalition_str == "{2}":
        return 0.1
    if coalition_str == "{3}":
        return 0.2
    if coalition_str == "{1,2}":
        return 0.3
    if coalition_str == "{1,3}":
        return 0.4
    if coalition_str == "{2,3}":
        return 0.35
    raise ValueError("Coalition not found")

#### Example computation

We illustrate how to apply the functions defined previously. Specifically, for $k=1,2$, $i=1, 2, 3$

In [6]:
from cgt_perezsechi.compute.shapley import exact

original = []
n = 3
shapley_value_mu1 = exact(n, mu1, original)
display(shapley_value_mu1)


shapley_value_mu2 = exact(n, mu2, original)
display(shapley_value_mu2)

[0.38333333333333336, 0.2333333333333333, 0.38333333333333336]

[0.35000000000000003, 0.27499999999999997, 0.375]

In [7]:
# Calculate mean Shapley values for each position
mean_shapley_values = []
for i in range(n):
    mean_value = (shapley_value_mu1[i] + shapley_value_mu2[i]) / 2
    mean_shapley_values.append(mean_value)

display(mean_shapley_values)

[0.3666666666666667, 0.25416666666666665, 0.37916666666666665]

In [8]:
from cgt_perezsechi.compute.grabisch import calculate_interaction_ij

original = []
n = 3

n_set = list(range(n))

interactions_mu1 = [[0.0 for j in n_set] for i in n_set]
for i in range(n):
    for j in range(n):
        if i != j:
            interactions_mu1[i][j] = calculate_interaction_ij(i, j, n_set, mu1, original)[2]

display(interactions_mu1)

interactions_mu2 = [[0.0 for j in n_set] for i in n_set]
for i in range(n):
    for j in range(n):
        if i != j:
            interactions_mu2[i][j] = calculate_interaction_ij(i, j, n_set, mu2, original)[2]
display(interactions_mu2)

[[0.0, -0.15, -0.04999999999999996],
 [-0.15, 0.0, 0.04999999999999996],
 [-0.04999999999999996, 0.04999999999999996, 0.0]]

[[0.0, 0.225, 0.22500000000000003],
 [0.225, 0.0, 0.27499999999999997],
 [0.22500000000000003, 0.27499999999999997, 0.0]]

In [9]:
# Calculate mean interactions for each position pair
mean_interactions = [[0.0 for j in range(n)] for i in range(n)]
for i in range(n):
    for j in range(n):
        if i != j:
            mean_interactions[i][j] = (interactions_mu1[i][j] + interactions_mu2[i][j]) / 2

display(mean_interactions)

[[0.0, 0.037500000000000006, 0.08750000000000004],
 [0.037500000000000006, 0.0, 0.16249999999999998],
 [0.08750000000000004, 0.16249999999999998, 0.0]]

- $\mathcal{R}^k(1, \mu^k)=Sh(\mu^k)$, so $\mathcal{R}^1(1, \mu^1)=\{0.3833, 0.2333, 0.3833\}$ and $\mathcal{R}^2(1, \mu^2)=\{0.3500, 0.2750, 0.3750\}$.
- $\mathcal{N}^*_i=mean(Sh_i(\mu^1),Sh_i(\mu^2))$, so $\mathcal{N}^*_1=0.3666$, $\mathcal{N}^*_2=0.2541$ and $\mathcal{N}^*_3=0.3791$.
- $\mathcal{R}^k(2, \mu^k)=I(\mu^k)$, so $\mathcal{R}^1(2, \mu^1)=\{\{0.0000, -0.1500, -0.0500\}, \{-0.1500, 0.0000, 0.0500\}, \{-0.0500, 0.0500, 0.000\}\}$ and $\mathcal{R}^2(2, \mu^2)=\{\{0.0000, 0.2250, 0.2250\}, \{0.2250, 0.0000, 0.2750\}, \{0.2250, 0.2750, 0.000\}$.
- $\mathcal{E}^*_{ij}=mean(I_{ij}(\mu^1),I_{ij}(\mu^2))$, so $\mathcal{E}^*=\big(\{\{0.0, 0.0375, 0.0875\}, \{0.0375, 0.0, 0.1625\}, \{0.0875, 0.1625, 0.0\}\}\big)$